In [1]:
import os
import sys

import torch
from torchvision import transforms
# Add project root to sys.path (one directory up from the notebook)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"Project root: {project_root}")
sys.path.insert(0, project_root)


from models.resnet50_backbone import ResNet50Backbone
from src.data_loader import ShanghaiTechDataModule

Project root: /Users/vittorio/Projects/uni/CV/CV-crowd-flow-estimation-


In [2]:

model = ResNet50Backbone()
model.load_state_dict(torch.load("../models/part_A_model.pth"))
model.eval()

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
test_loader = ShanghaiTechDataModule(
    "../data/ShanghaiTech",
    part="part_A",
).test_dataloader()


mae = mse = 0
with torch.no_grad():
    for img_batch, count_map_batch in test_loader:
        count_batch = count_map_batch.sum(dim=(1, 2)).unsqueeze(1).float()
        output_batch = model(img_batch)
        mae += torch.sum(torch.abs(output_batch - count_batch)).item()
        mse += torch.sum((output_batch - count_batch) ** 2).item()

print("MAE:", mae / len(test_loader.dataset))
print("RMSE:", (mse / len(test_loader.dataset))**0.5)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


MAE: 144.8908211844308
RMSE: 214.19119877873595
